In [1]:
import pandas as pd
import numpy as np

# Why the result is still not that accurate in the second part?
**Mayebe the features in the datset are not complicated enough for the model to learn the complete dynamic of the output. In this part, some new features will be added to see if they can improve the model's predictivity. Particularly, we will square some of the features to tell the model the history volatility of that feature.**

# Column name discription
1. C        -   consumption growth 
2. I        -   investment growth 
3. G        -   government spending growth 
4. IM       -   import ammount growth 
5. EX       -   export ammount growth 
6. unemploy -   unemployment rate (%)
7. LTGBY    -   Long Term Government Bond Yield(10 years) (%)
8. pop_44   -   population growth of age group 25-44 
9. pop_64   -   population growth of age group 45-64 
10. top10   -   top 10% pre-tax national income share (%)
11. temp    -   average tempreture (Fahrenheit)
12. inflation - inflation rate (%)
13. C2 - consumption volatility(squared consumption growth)
14. I2 - investment volatility(squared investment growth)
15. G2 - government spending volatility(squared government spending growth)
16. IM2 - import volatility(squared import growth)
17. EX2 - export volatility(squared export growth)
18. pop_44_2 - population of age group25-44 volatility(squared population growth of age group 25-44 )
19. pop_64_2 - population of age group45-64(squared population growth of age group 45-64 )

In [2]:
main_form = pd.read_csv("d:\econ1500\data\growth_volatility.csv")
# main_form.set_index('DATE', drop=False, inplace=True)
main_form = main_form.drop(['Year', 'DATE'], axis=1)

main_form_target = {}
main_form_features = {}
x_train = {}
x_val = {}
x_test = {}
y_train = {}
y_val = {}
y_test = {}

for i in main_form.columns:
    main_form_target[i] = main_form[[i]].shift(-1)
    main_form_features[i] = main_form
    # main_form_features[i] = main_form.drop([i], axis= 1)

    #determine train, val, and test size
    train_size = 0.7
    val_size = 0.15
    test_size = 0.15

    train_end = int(len(main_form[i]) * train_size)
    val_end = int(len(main_form[i]) * (train_size + val_size))
    #split data
    x_train[i] = main_form_features[i][:train_end]
    x_val[i] = main_form_features[i][train_end:val_end]
    x_test[i] = main_form_features[i][val_end:-1]
    y_train[i] = main_form_target[i][:train_end]
    y_val[i] = main_form_target[i][train_end:val_end]
    y_test[i] = main_form_target[i][val_end:-1]

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

for i in main_form.columns:
    imputer = IterativeImputer(estimator = LinearRegression(), random_state=42)
    X_impute = imputer.fit_transform(x_test[i])
    y_impute = imputer.fit_transform(y_test[i])
    x_test[i] = pd.DataFrame(data=X_impute, columns = x_test[i].columns)
    y_test[i] = pd.DataFrame(data=y_impute, columns = y_test[i].columns)

from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
X_train_prep = {}
X_val_prep = {}
X_test_prep = {}
for i in main_form.columns:

    std_ftrs = x_train[i].columns.tolist()
    preprocessor = ColumnTransformer(
        transformers = [('std', StandardScaler(), std_ftrs)])


    clf = Pipeline(steps = [('preprocessor', preprocessor)])

    X_train_prep[i] = clf.fit_transform(x_train[i])
    X_val_prep[i] = clf.transform(x_val[i])
    X_test_prep[i] = clf.transform(x_test[i])

from matplotlib import pylab as plt
X_train_prep['C']
X_train_prep['C'].shape
data = {}
for b, i in enumerate(main_form.columns):
    data[i] = []
    for a in range(X_train_prep['C'].shape[0]):
        data[i].append(X_train_prep['C'][a][b])


# Calculate baseline MSE for each feature using the mean value of each feature

In [3]:
avg = {}
sum = {}
for i in main_form.columns:
    sum[i] = []
    avg[i] = np.nanmean(main_form[i].tolist())
    for row in range(main_form.shape[0]):
        temp = np.square(main_form[i][row] - avg[i])
        sum[i].append(temp)
    sum[i] = np.nanmean(sum[i])
print(sum)

{'C': 0.00013942570792153423, 'I': 0.0018232935353420072, 'G': 0.00012762461405545976, 'IM': 0.001835747903518716, 'EX': 0.0018885648861745531, 'Temp': 3.114176567744687, 'inflation': 0.6445829784821459, 'unemploy': 2.9219570834887314, 'LTGBY': 8.438051192125313, 'pop_44': 7.522347708633277e-05, 'pop_64': 6.989088392758051e-05, 'top10': 19.076630320987654, 'C2': 6.261249693523105e-07, 'I2': 1.6433331292395095e-05, 'G2': 1.6112638968565786e-07, 'IM2': 2.9410881477357785e-05, 'EX2': 3.449307817023182e-05, 'pop_44_2': 1.1055536948347974e-07, 'pop_64_2': 6.042991708286192e-08}


In [4]:
pd.set_option('display.float_format', lambda x: '%.6f' % x)
MSE_score = pd.DataFrame.from_dict(sum, orient='index', columns = ['Baseline'])
MSE_score

,Baseline
C,0.000139
I,0.001823
G,0.000128
IM,0.001836
EX,0.001889
Temp,3.114177
inflation,0.644583
unemploy,2.921957
LTGBY,8.438051
pop_44,0.000075


# Ridge Model
**Hyper-parameter to be tuned: alpha value**

In [5]:
from sklearn.model_selection import ParameterGrid
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

alpha_values = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800]
param_grid = {'regressor__alpha': alpha_values}

train_rmse_score_ridge = np.zeros(len(ParameterGrid(param_grid)))
val_rmse_score_ridge = np.zeros(len(ParameterGrid(param_grid)))

test_rmse_scores_ridge = {}
final_models_ridge = {}
for i in main_form.columns:
    models = []
    base_rmse_test_ridge = []
    base_rmse_val_ridge = []
    val_rmse_scores_ridge = []

    print('---------------------------------------')
    print('Ridge model to predict', i)
    for p, params in enumerate(ParameterGrid(param_grid)):
            params = ParameterGrid(param_grid)[p]
            print('   ',params) 
            
            model = Ridge(alpha=params['regressor__alpha'], solver = 'saga' ,random_state=42, max_iter=10000)
            
            model.fit(X_train_prep[i],y_train[i]) 
            models.append(model) 
            
            y_train_pred = model.predict(X_train_prep[i])
            train_rmse_score_ridge[p] = mean_squared_error(y_train[i], y_train_pred)
            y_val_pred = model.predict(X_val_prep[i])
            val_rmse_score_ridge[p] = mean_squared_error(y_val[i], y_val_pred)
            print('   ', train_rmse_score_ridge[p],val_rmse_score_ridge[p])
    

   
    print('best model parameters:',ParameterGrid(param_grid)[np.argmin(val_rmse_score_ridge)])
    print('corresponding validation score:',np.min(val_rmse_score_ridge))
 

    
    final_models_ridge[i] = (models[np.argmin(val_rmse_score_ridge)])
    
    y_test_pred = final_models_ridge[i].predict(X_test_prep[i])
        
    test_rmse_scores_ridge[i] = mean_squared_error(y_test[i], y_test_pred)
    print('test score:',test_rmse_scores_ridge[i])
    

---------------------------------------
Ridge model to predict C
    {'regressor__alpha': 0.0001}
    4.0095706424184545e-05 6.86418972607705e-05
    {'regressor__alpha': 0.001}
    4.0095706947086504e-05 6.86403335323549e-05
    {'regressor__alpha': 0.01}
    4.009571350036807e-05 6.862396235561167e-05
    {'regressor__alpha': 0.1}
    4.009583752888424e-05 6.846245049783147e-05
    {'regressor__alpha': 1}
    4.010137556805075e-05 6.71551917596958e-05
    {'regressor__alpha': 10}
    4.02703676028841e-05 6.303146001668107e-05
    {'regressor__alpha': 100}
    4.209307595782365e-05 6.492011970657333e-05
    {'regressor__alpha': 150}
    4.28990210989054e-05 6.745374566187986e-05
    {'regressor__alpha': 200}
    4.3612730782424376e-05 6.988960166764655e-05
    {'regressor__alpha': 250}
    4.425623236841482e-05 7.216669932771116e-05
    {'regressor__alpha': 300}
    4.484364759417589e-05 7.427922724270482e-05
    {'regressor__alpha': 350}
    4.5385063012222136e-05 7.623784755943287e-

**Store the MSE results in the table**

In [6]:
MSE_ridge = pd.DataFrame.from_dict(test_rmse_scores_ridge, orient='index', columns=['Ridge'])
MSE_score = MSE_score.join(MSE_ridge)
MSE_score


,Baseline,Ridge
C,0.000139,0.000467
I,0.001823,0.003265
G,0.000128,0.000214
IM,0.001836,0.002395
EX,0.001889,0.002876
Temp,3.114177,2.061138
inflation,0.644583,0.672356
unemploy,2.921957,7.204396
LTGBY,8.438051,0.656942
pop_44,0.000075,0.000130


**Store the best tuned hyper-parameter into the model**

In [7]:
final_models_ridge

{'C': Ridge(alpha=10, max_iter=10000, random_state=42, solver='saga'),
 'I': Ridge(alpha=100, max_iter=10000, random_state=42, solver='saga'),
 'G': Ridge(alpha=10, max_iter=10000, random_state=42, solver='saga'),
 'IM': Ridge(alpha=300, max_iter=10000, random_state=42, solver='saga'),
 'EX': Ridge(alpha=800, max_iter=10000, random_state=42, solver='saga'),
 'Temp': Ridge(alpha=100, max_iter=10000, random_state=42, solver='saga'),
 'inflation': Ridge(alpha=500, max_iter=10000, random_state=42, solver='saga'),
 'unemploy': Ridge(alpha=0.0001, max_iter=10000, random_state=42, solver='saga'),
 'LTGBY': Ridge(alpha=1, max_iter=10000, random_state=42, solver='saga'),
 'pop_44': Ridge(alpha=150, max_iter=10000, random_state=42, solver='saga'),
 'pop_64': Ridge(alpha=800, max_iter=10000, random_state=42, solver='saga'),
 'top10': Ridge(alpha=0.0001, max_iter=10000, random_state=42, solver='saga'),
 'C2': Ridge(alpha=150, max_iter=10000, random_state=42, solver='saga'),
 'I2': Ridge(alpha=1, m

# Lasso Model
**Hyper-parameter to be tuned: alpha values**

In [8]:
from sklearn.model_selection import ParameterGrid
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

alpha_values = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800]
param_grid = {'regressor__alpha': alpha_values}

train_rmse_score_ridge = np.zeros(len(ParameterGrid(param_grid)))
val_rmse_score_ridge = np.zeros(len(ParameterGrid(param_grid)))

param_grid = {'regressor__alpha': alpha_values}
test_rmse_scores_lasso = {}
final_models_lasso = {}
for i in main_form.columns:
    models = []
    base_rmse_test_lasso = []
    base_rmse_val_lasso = []
    val_rmse_scores_lasso = []

    print('---------------------------------------')
    print('Lasso model to predict', i)
    for p, params in enumerate(ParameterGrid(param_grid)):
        params = ParameterGrid(param_grid)[p]
        print('   ',params) 
            
        model = Lasso(alpha=params['regressor__alpha'], selection = 'random' ,random_state=42, max_iter=100000)
            
        model.fit(X_train_prep[i],y_train[i]) 
        models.append(model) 
            
        y_train_pred = model.predict(X_train_prep[i])
        train_rmse_score_ridge[p] = mean_squared_error(y_train[i], y_train_pred)
        y_val_pred = model.predict(X_val_prep[i])
        val_rmse_score_ridge[p] = mean_squared_error(y_val[i], y_val_pred)
            
        print('   ', train_rmse_score_ridge[p],val_rmse_score_ridge[p])
        

    final_models_lasso[i] = (models[np.argmin(val_rmse_score_ridge)])
    print('best model parameters:',ParameterGrid(param_grid)[np.argmin(val_rmse_score_ridge)])
    print('corresponding validation score:',np.min(val_rmse_score_ridge))
 

    
    
    
    y_test_pred = final_models_lasso[i].predict(X_test_prep[i])
        
    test_rmse_scores_lasso[i] = mean_squared_error(y_test[i], y_test_pred)
    print('test score:',test_rmse_scores_lasso[i])
    

---------------------------------------
Lasso model to predict C
    {'regressor__alpha': 0.0001}
    4.05726173123567e-05 6.421723408379838e-05
    {'regressor__alpha': 0.001}
    4.627802547042013e-05 6.867994316590825e-05
    {'regressor__alpha': 0.01}
    6.176419107675654e-05 0.00012779565283764272
    {'regressor__alpha': 0.1}
    6.176419107675654e-05 0.00012779565283764272
    {'regressor__alpha': 1}
    6.176419107675654e-05 0.00012779565283764272
    {'regressor__alpha': 10}
    6.176419107675654e-05 0.00012779565283764272
    {'regressor__alpha': 100}
    6.176419107675654e-05 0.00012779565283764272
    {'regressor__alpha': 150}
    6.176419107675654e-05 0.00012779565283764272
    {'regressor__alpha': 200}
    6.176419107675654e-05 0.00012779565283764272
    {'regressor__alpha': 250}
    6.176419107675654e-05 0.00012779565283764272
    {'regressor__alpha': 300}
    6.176419107675654e-05 0.00012779565283764272
    {'regressor__alpha': 350}
    6.176419107675654e-05 0.00012779

**Store the MSE results into the table**

In [9]:
lasso_regre = pd.DataFrame.from_dict(test_rmse_scores_lasso, orient = 'index', columns=['Lasso'])
MSE_score = MSE_score.join(lasso_regre)
MSE_score

,Baseline,Ridge,Lasso
C,0.000139,0.000467,0.000448
I,0.001823,0.003265,0.002438
G,0.000128,0.000214,0.000214
IM,0.001836,0.002395,0.004427
EX,0.001889,0.002876,0.003137
Temp,3.114177,2.061138,1.020823
inflation,0.644583,0.672356,0.677697
unemploy,2.921957,7.204396,6.668867
LTGBY,8.438051,0.656942,0.592230
pop_44,0.000075,0.000130,0.000127


**Store the best tuned hyper-parameter into the model**

In [10]:
final_models_lasso

{'C': Lasso(alpha=0.0001, max_iter=100000, random_state=42, selection='random'),
 'I': Lasso(alpha=0.01, max_iter=100000, random_state=42, selection='random'),
 'G': Lasso(alpha=0.0001, max_iter=100000, random_state=42, selection='random'),
 'IM': Lasso(alpha=0.001, max_iter=100000, random_state=42, selection='random'),
 'EX': Lasso(alpha=0.01, max_iter=100000, random_state=42, selection='random'),
 'Temp': Lasso(alpha=0.1, max_iter=100000, random_state=42, selection='random'),
 'inflation': Lasso(alpha=0.1, max_iter=100000, random_state=42, selection='random'),
 'unemploy': Lasso(alpha=0.001, max_iter=100000, random_state=42, selection='random'),
 'LTGBY': Lasso(alpha=0.01, max_iter=100000, random_state=42, selection='random'),
 'pop_44': Lasso(alpha=0.001, max_iter=100000, random_state=42, selection='random'),
 'pop_64': Lasso(alpha=0.01, max_iter=100000, random_state=42, selection='random'),
 'top10': Lasso(alpha=0.01, max_iter=100000, random_state=42, selection='random'),
 'C2': La

# XGBoost Model
**Hyper-parameter to be tuned: learning rate, max_depth, n_estimators**

In [11]:
#XGBoost
import xgboost
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
import warnings


warnings.filterwarnings('ignore', message='`eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.', category=UserWarning)


param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [1 , 2, 3, 4, 5],
    "missing": [np.nan], 
    'n_estimators': [25, 50, 100, 200], 
    'min_child_weight': [0.5, 1, 1.5, 2]
    
}


test_rmse_scores_xgb = {}
best_models_xgb = {}
for i in main_form.columns:
    print('---------------------------------------')
    print('XGBoost model to predict', i)

    model = XGBRegressor(eval_metric="rmse")
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_root_mean_squared_error', cv=2)
    grid_search.fit(X_train_prep[i], y_train[i])

    
    best_model = grid_search.best_estimator_
    best_models_xgb[i] = best_model

   
    y_train_pred = best_model.predict(X_train_prep[i])
    y_val_pred = best_model.predict(X_val_prep[i])
    y_test_pred = best_model.predict(X_test_prep[i])
    
    train_rmse = mean_squared_error(y_train[i], y_train_pred)
    val_rmse = mean_squared_error(y_val[i], y_val_pred)
    test_rmse = mean_squared_error(y_test[i], y_test_pred)

    test_rmse_scores_xgb[i] = test_rmse


    print('Train RMSE:', train_rmse)
    print('Validation RMSE:', val_rmse)
    print('Test RMSE:', test_rmse)

    print('Best model parameters:', grid_search.best_params_)
    print('Corresponding validation score:', grid_search.best_score_)
    



    

---------------------------------------
XGBoost model to predict C
Train RMSE: 1.5568249321093855e-05
Validation RMSE: 8.673292624572612e-05
Test RMSE: 0.0005159714077664484
Best model parameters: {'learning_rate': 0.2, 'max_depth': 2, 'min_child_weight': 0.5, 'missing': nan, 'n_estimators': 50}
Corresponding validation score: -0.008370962399915778
---------------------------------------
XGBoost model to predict I
Train RMSE: 0.001374003984261499
Validation RMSE: 0.0012809986033068602
Test RMSE: 0.0017166975356806443
Best model parameters: {'learning_rate': 0.2, 'max_depth': 1, 'min_child_weight': 1.5, 'missing': nan, 'n_estimators': 25}
Corresponding validation score: -0.04354568840612455
---------------------------------------
XGBoost model to predict G
Train RMSE: 0.00010889638045344403
Validation RMSE: 0.00011069594018980994
Test RMSE: 0.00010092662274516089
Best model parameters: {'learning_rate': 0.2, 'max_depth': 1, 'min_child_weight': 0.5, 'missing': nan, 'n_estimators': 25}
Co

**Store the MSE results into the table**

In [12]:
xgb = pd.DataFrame.from_dict(test_rmse_scores_xgb, orient = 'index', columns=['XGBoost'])

MSE_score = MSE_score.join(xgb)

MSE_score

,Baseline,Ridge,Lasso,XGBoost
C,0.000139,0.000467,0.000448,0.000516
I,0.001823,0.003265,0.002438,0.001717
G,0.000128,0.000214,0.000214,0.000101
IM,0.001836,0.002395,0.004427,0.001770
EX,0.001889,0.002876,0.003137,0.002008
Temp,3.114177,2.061138,1.020823,5.642255
inflation,0.644583,0.672356,0.677697,0.472086
unemploy,2.921957,7.204396,6.668867,2.877551
LTGBY,8.438051,0.656942,0.592230,2.625457
pop_44,0.000075,0.000130,0.000127,0.000111


**Store the best tuned hyper-parameter into the model**

In [13]:
best_models_xgb

{'C': XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='rmse', feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=0.5, missing=nan, monotone_constraints=None,
              n_estimators=50, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...),
 'I': XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='r

# SVR model
**Hyper parameter to be tuned: C value, gamma value and kernel type**

In [14]:
from sklearn.svm import SVR


param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [0.01, 0.1, 0.5],
    'kernel': ['rbf', 'linear']
}


test_rmse_scores_svr = {}
best_models_svr = {}
for i in main_form.columns:
    print('---------------------------------------')
    print('SVR model to predict', i)

    model = SVR()
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_root_mean_squared_error', cv=2)
    grid_search.fit(X_train_prep[i], y_train[i].values.ravel())

    
    best_model = grid_search.best_estimator_
    best_models_svr[i] = best_model

   
    y_train_pred = best_model.predict(X_train_prep[i])
    y_val_pred = best_model.predict(X_val_prep[i])
    y_test_pred = best_model.predict(X_test_prep[i])
    
    train_rmse = mean_squared_error(y_train[i].values.ravel(), y_train_pred)
    val_rmse = mean_squared_error(y_val[i].values.ravel(), y_val_pred)
    test_rmse = mean_squared_error(y_test[i].values.ravel(), y_test_pred)

    test_rmse_scores_svr[i] = test_rmse


    print('Train RMSE:', train_rmse)
    print('Validation RMSE:', val_rmse)
    print('Test RMSE:', test_rmse)

    print('Best model parameters:', grid_search.best_params_)
    print('Corresponding validation score:', grid_search.best_score_)
    



    

---------------------------------------
SVR model to predict C
Train RMSE: 6.757122272620369e-05
Validation RMSE: 0.00017263164410076946
Test RMSE: 0.0005738496222061725
Best model parameters: {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
Corresponding validation score: -0.008197565194482344
---------------------------------------
SVR model to predict I
Train RMSE: 0.0017851364455688278
Validation RMSE: 0.0013811513541344207
Test RMSE: 0.0018043050214501559
Best model parameters: {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
Corresponding validation score: -0.0424016117722705
---------------------------------------
SVR model to predict G
Train RMSE: 0.0001399652865093335
Validation RMSE: 0.0001291433940589671
Test RMSE: 9.49161366479718e-05
Best model parameters: {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
Corresponding validation score: -0.011810107601942519
---------------------------------------
SVR model to predict IM
Train RMSE: 0.001415714915931433
Validation RMSE: 0.0027940603921849617

**Store the MSE results into the table**

In [15]:
svr = pd.DataFrame.from_dict(test_rmse_scores_svr, orient = 'index', columns=['SVR'])

MSE_score = MSE_score.join(svr)

MSE_score

,Baseline,Ridge,Lasso,XGBoost,SVR
C,0.000139,0.000467,0.000448,0.000516,0.000574
I,0.001823,0.003265,0.002438,0.001717,0.001804
G,0.000128,0.000214,0.000214,0.000101,0.000095
IM,0.001836,0.002395,0.004427,0.001770,0.002990
EX,0.001889,0.002876,0.003137,0.002008,0.002993
Temp,3.114177,2.061138,1.020823,5.642255,0.718994
inflation,0.644583,0.672356,0.677697,0.472086,0.474940
unemploy,2.921957,7.204396,6.668867,2.877551,6.149989
LTGBY,8.438051,0.656942,0.592230,2.625457,0.399430
pop_44,0.000075,0.000130,0.000127,0.000111,0.000196


**Store the best tuned hyper-parameter into the model**

In [16]:
best_models_svr

{'C': SVR(C=0.1, gamma=0.01),
 'I': SVR(C=0.1, gamma=0.01),
 'G': SVR(C=0.1, gamma=0.01),
 'IM': SVR(C=1, gamma=0.01),
 'EX': SVR(C=0.1, gamma=0.01),
 'Temp': SVR(C=0.1, gamma=0.01, kernel='linear'),
 'inflation': SVR(C=1, gamma=0.01),
 'unemploy': SVR(C=0.1, gamma=0.01, kernel='linear'),
 'LTGBY': SVR(C=1, gamma=0.01, kernel='linear'),
 'pop_44': SVR(C=0.1, gamma=0.01),
 'pop_64': SVR(C=0.1, gamma=0.01),
 'top10': SVR(C=10, gamma=0.01, kernel='linear'),
 'C2': SVR(C=0.1, gamma=0.01),
 'I2': SVR(C=0.1, gamma=0.01),
 'G2': SVR(C=0.1, gamma=0.01),
 'IM2': SVR(C=0.1, gamma=0.01),
 'EX2': SVR(C=0.1, gamma=0.01),
 'pop_44_2': SVR(C=0.1, gamma=0.01),
 'pop_64_2': SVR(C=0.1, gamma=0.01)}

# Random Forest
**Hyper-parameters to be tuned: max features, max depth**

In [17]:
from sklearn.ensemble import RandomForestRegressor

param_grid = {
              'max_features': [None,1, 3, 10, 20], 
              'max_depth': [None,1, 3, 10, 20]
              } 


test_rmse_scores_rf = {}
best_models_rf = {}
for i in main_form.columns:
    print('---------------------------------------')
    print('RF model to predict', i)

    model = RandomForestRegressor()
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_root_mean_squared_error', cv=2)
    grid_search.fit(X_train_prep[i], y_train[i].values.ravel())

    
    best_model = grid_search.best_estimator_
    best_models_rf[i] = best_model

   
    y_train_pred = best_model.predict(X_train_prep[i])
    y_val_pred = best_model.predict(X_val_prep[i])
    y_test_pred = best_model.predict(X_test_prep[i])
    
    train_rmse = mean_squared_error(y_train[i].values.ravel(), y_train_pred)
    val_rmse = mean_squared_error(y_val[i].values.ravel(), y_val_pred)
    test_rmse = mean_squared_error(y_test[i].values.ravel(), y_test_pred)

    test_rmse_scores_rf[i] = test_rmse


    print('Train RMSE:', train_rmse)
    print('Validation RMSE:', val_rmse)
    print('Test RMSE:', test_rmse)

    print('Best model parameters:', grid_search.best_params_)
    print('Corresponding validation score:', grid_search.best_score_)

---------------------------------------
RF model to predict C
Train RMSE: 6.497124707468311e-06
Validation RMSE: 9.513251600120706e-05
Test RMSE: 0.0004920335724119294
Best model parameters: {'max_depth': 20, 'max_features': 1}
Corresponding validation score: -0.007383289119859464
---------------------------------------
RF model to predict I
Train RMSE: 0.0012656714896559268
Validation RMSE: 0.0013495060695211888
Test RMSE: 0.0015838212240157437
Best model parameters: {'max_depth': 3, 'max_features': 3}
Corresponding validation score: -0.04229543281581164
---------------------------------------
RF model to predict G
Train RMSE: 3.3131244921400176e-05
Validation RMSE: 9.809317088026612e-05
Test RMSE: 7.686568640630482e-05
Best model parameters: {'max_depth': 10, 'max_features': 1}
Corresponding validation score: -0.011600402254378956
---------------------------------------
RF model to predict IM
Train RMSE: 0.00020667049301364008
Validation RMSE: 0.0025277699119509037
Test RMSE: 0.00193

**Store the MSE results into the table**

In [18]:
rf = pd.DataFrame.from_dict(test_rmse_scores_rf, orient = 'index', columns=['Random Forest'])

MSE_score = MSE_score.join(rf)

MSE_score

,Baseline,Ridge,Lasso,XGBoost,SVR,Random Forest
C,0.000139,0.000467,0.000448,0.000516,0.000574,0.000492
I,0.001823,0.003265,0.002438,0.001717,0.001804,0.001584
G,0.000128,0.000214,0.000214,0.000101,0.000095,0.000077
IM,0.001836,0.002395,0.004427,0.001770,0.002990,0.001939
EX,0.001889,0.002876,0.003137,0.002008,0.002993,0.002729
Temp,3.114177,2.061138,1.020823,5.642255,0.718994,3.661053
inflation,0.644583,0.672356,0.677697,0.472086,0.474940,0.507865
unemploy,2.921957,7.204396,6.668867,2.877551,6.149989,3.127739
LTGBY,8.438051,0.656942,0.592230,2.625457,0.399430,1.457223
pop_44,0.000075,0.000130,0.000127,0.000111,0.000196,0.000109


**Store the best tuned hyper-parameter into the model**

In [19]:
best_models_rf

{'C': RandomForestRegressor(max_depth=20, max_features=1),
 'I': RandomForestRegressor(max_depth=3, max_features=3),
 'G': RandomForestRegressor(max_depth=10, max_features=1),
 'IM': RandomForestRegressor(max_depth=20, max_features=3),
 'EX': RandomForestRegressor(max_depth=20, max_features=1),
 'Temp': RandomForestRegressor(max_depth=3, max_features=20),
 'inflation': RandomForestRegressor(max_depth=3, max_features=20),
 'unemploy': RandomForestRegressor(max_features=None),
 'LTGBY': RandomForestRegressor(max_depth=10, max_features=None),
 'pop_44': RandomForestRegressor(max_depth=10, max_features=1),
 'pop_64': RandomForestRegressor(max_depth=1, max_features=1),
 'top10': RandomForestRegressor(max_depth=10, max_features=None),
 'C2': RandomForestRegressor(max_depth=10, max_features=1),
 'I2': RandomForestRegressor(max_depth=3, max_features=1),
 'G2': RandomForestRegressor(max_depth=1, max_features=3),
 'IM2': RandomForestRegressor(max_depth=1, max_features=None),
 'EX2': RandomForest

# 1D-Convolutional Neural Network Model(1D-CNN)
**Hyper-parameter to be tuned: number of filters, kernel size**

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier, KerasRegressor

**Preprocess the train, val, test data to fit the shape requirement of 1D-CNN**

In [21]:
time_steps = 10
X={}
y={}
X_val_NN = {}
y_val_NN = {}
X_test_NN = {}
y_test_NN = {}

for i in main_form.columns:
    # X[i]={}
    # y[i]={}
    
    X[i] = []
    y[i] = []
    X_val_NN[i] = []
    y_val_NN[i] = []
    X_test_NN[i] = []
    y_test_NN[i] = []
    y_train_list = y_train[i][i].tolist()
    y_val_list = y_val[i][i].tolist()
    y_test_list = y_test[i][i].tolist()
    for time in range(time_steps, len(X_train_prep[i])):
        X[i].append(X_train_prep[i][time-time_steps:time])  
        y[i].append(y_train_list[time])  
    for time in range(time_steps, len(X_val_prep[i])):
        X_val_NN[i].append(X_val_prep[i][time-time_steps:time])  
        y_val_NN[i].append(y_val_list[time])
    for time in range(time_steps, len(X_test_prep[i])):
        X_test_NN[i].append(X_test_prep[i][time-time_steps:time])  
        y_test_NN[i].append(y_test_list[time])

    X[i] = np.array(X[i])
    y[i] = np.array(y[i])
    X_val_NN[i] = np.array(X_val_NN[i])
    y_val_NN[i] = np.array(y_val_NN[i])
    X_test_NN[i] = np.array(X_test_NN[i])
    y_test_NN[i] = np.array(y_test_NN[i])

In [22]:
# X['I'].shape

In [23]:
# X_test_NN['I']

**Train 1D-CNN**

In [24]:
from tensorflow.keras import backend as K
import warnings
warnings.filterwarnings('ignore')

# Your code here (e.g., model training, evaluation, etc.)
best_models_CNN = {}
CNN_loss = {}

def create_model(filters=256, kernel_size=3):
    def rmse(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))
    
    model = Sequential([
        Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=(X[next(iter(X))].shape[1], X[next(iter(X))].shape[2])),
        Conv1D(filters=filters, kernel_size=kernel_size, activation='relu'),
        Flatten(),
        Dense(50, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss=rmse)
    return model

model = KerasRegressor(build_fn=create_model, epochs=15, verbose=0, filters=128, kernel_size=3)

param_grid = {
    'filters': [32, 64, 128, 256, 512],
    'kernel_size': [1, 2, 3, 4, 5]
}

grid_result = {}
for i in main_form.columns:
    print('Predicting', i, 'using CNN with GridSearchCV')
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=2, scoring='neg_mean_squared_error')
    grid_result[i] = grid.fit(X[i], y[i])
    print(f'Best Score: {grid_result[i].best_score_} for {i} using {grid_result[i].best_params_}')

    best_model = grid_result[i].best_estimator_
    best_models_CNN[i] = best_model
    test_pred= best_models_CNN[i].predict(X_test_NN[i])
    test_rmse = mean_squared_error(y_test_NN[i], test_pred)
    CNN_loss[i] = test_rmse
    print('test loss is', CNN_loss[i])
    print()
    






Predicting C using CNN with GridSearchCV
Best Score: -6.988503892856117e-05 for C using {'filters': 512, 'kernel_size': 2}
test loss is 0.0006951003221539119

Predicting I using CNN with GridSearchCV
Best Score: -0.0018685307666251186 for I using {'filters': 512, 'kernel_size': 1}
test loss is 0.0023608956873486754

Predicting G using CNN with GridSearchCV
Best Score: -0.00013193999883174393 for G using {'filters': 256, 'kernel_size': 2}
test loss is 0.0005517802546997616

Predicting IM using CNN with GridSearchCV
Best Score: -0.0017537225174180857 for IM using {'filters': 512, 'kernel_size': 3}
test loss is 0.004921001377342224

Predicting EX using CNN with GridSearchCV
Best Score: -0.0018960924203387129 for EX using {'filters': 512, 'kernel_size': 4}
test loss is 0.0038494216245426698

Predicting Temp using CNN with GridSearchCV
Best Score: -92.09299963621909 for Temp using {'filters': 512, 'kernel_size': 1}
test loss is 1230.031497235342

Predicting inflation using CNN with GridSear

**Store the MSE results into the table**

In [25]:
NN_loss = pd.DataFrame.from_dict(CNN_loss, orient = 'index', columns = ['CNN'])
MSE_score = MSE_score.join(NN_loss)
MSE_score

,Baseline,Ridge,Lasso,XGBoost,SVR,Random Forest,CNN
C,0.000139,0.000467,0.000448,0.000516,0.000574,0.000492,0.000695
I,0.001823,0.003265,0.002438,0.001717,0.001804,0.001584,0.002361
G,0.000128,0.000214,0.000214,0.000101,0.000095,0.000077,0.000552
IM,0.001836,0.002395,0.004427,0.001770,0.002990,0.001939,0.004921
EX,0.001889,0.002876,0.003137,0.002008,0.002993,0.002729,0.003849
Temp,3.114177,2.061138,1.020823,5.642255,0.718994,3.661053,1230.031497
inflation,0.644583,0.672356,0.677697,0.472086,0.474940,0.507865,5.844558
unemploy,2.921957,7.204396,6.668867,2.877551,6.149989,3.127739,34.672524
LTGBY,8.438051,0.656942,0.592230,2.625457,0.399430,1.457223,120.196504
pop_44,0.000075,0.000130,0.000127,0.000111,0.000196,0.000109,0.001014


**Store the best tuned hyper-parameter into the model**

In [26]:
best_models_CNN

{'C': KerasRegressor(
 	model=None
 	build_fn=<function create_model at 0x00000260C8156DE0>
 	warm_start=False
 	random_state=None
 	optimizer=rmsprop
 	loss=None
 	metrics=None
 	batch_size=None
 	validation_batch_size=None
 	verbose=0
 	callbacks=None
 	validation_split=0.0
 	shuffle=True
 	run_eagerly=False
 	epochs=15
 	filters=512
 	kernel_size=2
 ),
 'I': KerasRegressor(
 	model=None
 	build_fn=<function create_model at 0x00000260C8156DE0>
 	warm_start=False
 	random_state=None
 	optimizer=rmsprop
 	loss=None
 	metrics=None
 	batch_size=None
 	validation_batch_size=None
 	verbose=0
 	callbacks=None
 	validation_split=0.0
 	shuffle=True
 	run_eagerly=False
 	epochs=15
 	filters=512
 	kernel_size=1
 ),
 'G': KerasRegressor(
 	model=None
 	build_fn=<function create_model at 0x00000260C8156DE0>
 	warm_start=False
 	random_state=None
 	optimizer=rmsprop
 	loss=None
 	metrics=None
 	batch_size=None
 	validation_batch_size=None
 	verbose=0
 	callbacks=None
 	validation_split=0.0
 	shuff

# Let's have a look at the result here:

In [27]:
MSE_score

,Baseline,Ridge,Lasso,XGBoost,SVR,Random Forest,CNN
C,0.000139,0.000467,0.000448,0.000516,0.000574,0.000492,0.000695
I,0.001823,0.003265,0.002438,0.001717,0.001804,0.001584,0.002361
G,0.000128,0.000214,0.000214,0.000101,0.000095,0.000077,0.000552
IM,0.001836,0.002395,0.004427,0.001770,0.002990,0.001939,0.004921
EX,0.001889,0.002876,0.003137,0.002008,0.002993,0.002729,0.003849
Temp,3.114177,2.061138,1.020823,5.642255,0.718994,3.661053,1230.031497
inflation,0.644583,0.672356,0.677697,0.472086,0.474940,0.507865,5.844558
unemploy,2.921957,7.204396,6.668867,2.877551,6.149989,3.127739,34.672524
LTGBY,8.438051,0.656942,0.592230,2.625457,0.399430,1.457223,120.196504
pop_44,0.000075,0.000130,0.000127,0.000111,0.000196,0.000109,0.001014


**Although part of the features' MSEs are still higher than their baseline, most of their MSE scores decrease. That is a good sign, indicating that we can get a more accurate result by incorporating more complicated features into the model.**